# Tool Calling with Azure OpenAI: gpt-4o-mini

In [1]:
from openai import AzureOpenAI

In [2]:
client = AzureOpenAI()

In [3]:
import requests,json
def get_current_weather(city:str)->dict:
    """ can be used to get/fetch current weather information for a city name
    """
    api_key = "6a8b0ac166a37e2b7a38e64416b3c3fe"

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = response.content.decode()
    response = json.loads(response)
    output = {"City Name":city,"weather":response["weather"][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}

    return output

In [4]:
get_current_weather("mumbai")

{'City Name': 'mumbai',
 'weather': 'broken clouds',
 'temperature': 299.7,
 'unit': 'kelvin'}

### Tool Integration with LLM
- metadata
    - name of tool/function
    - description of the tool/function
    - arguments details - arguments and its description for the tool/function

In [11]:
tools = [{
    "type":"function",
    "function":{
        "name":"get_current_weather",
        "description":"this function can be used to get the current weather information for any given city.",
        "parameters":{
            "type":"object",
            "properties":{"city":{"type":"string","description":"name of city e.g. mumbai, new york"}},
        },
        "required":['city'],
    }}
]

In [18]:
tool_map={"get_current_weather":get_current_weather}

def generate_response(prompt):

    messages = [{"role":"system","content":"you are a helpful assistant"},
                {"role":"user",'content':prompt}]
    
    first_response = client.chat.completions.create(model='gpt4o',
                                                    messages=messages,
                                                    temperature=0.2,
                                                    tools=tools,tool_choice='auto')
    if first_response.choices[0].message.tool_calls:
        # do something
        print("LLM decided to trigger tool call",first_response.choices[0].message.tool_calls)
        tool_calls = first_response.choices[0].message.tool_calls
        messages.append(first_response.choices[0].message)
        for tool in tool_calls:
            tool_name=tool.function.name
            tool_args = json.loads(tool.function.arguments)
            fun_to_ex = tool_map[tool_name]
            tool_output = fun_to_ex(**tool_args)
            messages.append({"tool_call_id":tool.id,"role":"tool",
                            "name":tool_name,"content":json.dumps(tool_output)})
        second_response = client.chat.completions.create(model='gpt4o',messages=messages,
                                                         temperature=0.1)
        return second_response.choices[0].message.content

    else:
        return first_response.choices[0].message.content

In [19]:
generate_response("What is Artificial Intelligence")

'Artificial Intelligence (AI) refers to the simulation of human intelligence processes by computer systems. These processes include learning (the acquisition of information and rules for using it), reasoning (using rules to reach approximate or definite conclusions), and self-correction. AI can be categorized into two main types:\n\n1. **Narrow AI (Weak AI)**: This type of AI is designed and trained for a specific task. Examples include virtual assistants like Siri and Alexa, recommendation systems, and image recognition software.\n\n2. **General AI (Strong AI)**: This type of AI would have the ability to understand, learn, and apply intelligence to solve any problem, much like a human. General AI remains largely theoretical and has not yet been achieved.\n\nAI technologies encompass various subfields, including:\n\n- **Machine Learning**: A subset of AI that enables systems to learn from data and improve their performance over time without being explicitly programmed.\n- **Natural Lan

In [20]:
generate_response("what is the current weather in delhi today?")

LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_eU5zGoXvBHoPvPFHrenNC35b', function=Function(arguments='{"city":"delhi"}', name='get_current_weather'), type='function')]


'The current weather in Delhi today is clear sky with a temperature of approximately 305.25 K (which is about 32.1°C).'